In [2]:
import numpy as np
import pandas as pd
import scipy.linalg as la
import scipy.optimize as opt
import copy
np.set_printoptions(threshold=np.inf, suppress=True, formatter={'float': '{: 0.8f}'.format}, linewidth=75)

In [3]:
test_matrix = np.array([[1, 2], [3, 4]])
matr2 = np.array([[5, 6], [7, 8]])
big_test_matrix = la.block_diag(test_matrix, test_matrix)
readable_big_test_matrix = pd.DataFrame(big_test_matrix)
print(type(big_test_matrix))

<class 'numpy.ndarray'>


In [4]:
def kron(A, B):
    return la.kron(A, B)

In [5]:
def kronsum(A, B):
    if A.shape[0] != A.shape[1]:
        raise ValueError('A is not square')

    if B.shape[0] != B.shape[1]:
        raise ValueError('B is not square')
    
    L = kron(A, np.eye(B.shape[0]))
    R = kron(np.eye(A.shape[0]), B)
    
    return L+R

In [6]:
print(kronsum(test_matrix, matr2))

[[ 6.00000000  6.00000000  2.00000000  0.00000000]
 [ 7.00000000  9.00000000  0.00000000  2.00000000]
 [ 3.00000000  0.00000000  9.00000000  6.00000000]
 [ 0.00000000  3.00000000  7.00000000  12.00000000]]


In [7]:
print(kronsum(big_test_matrix[:2, :2], big_test_matrix[2:4, 2:4]))

[[ 2.00000000  2.00000000  2.00000000  0.00000000]
 [ 3.00000000  5.00000000  0.00000000  2.00000000]
 [ 3.00000000  0.00000000  5.00000000  2.00000000]
 [ 0.00000000  3.00000000  3.00000000  8.00000000]]


In [8]:
n = 3
p_num = 200
eps_G = 10 ** (-8)
eps_Phi = 10 ** (-6)
eps_p_i = 10 ** (-6)

In [9]:
# Входной BMAP
matrD_0 = np.array([[-86., 0.01], [0.02, -2.76]]) / 7
matrD = np.array([[85, 0.99], [0.2, 2.54]]) / 7
matrD_k = [matrD_0]

W_ = matrD_0.shape[0]
W = W_ - 1

q = 0.8
for k in range(1, n+1):
    matrD_k.append(matrD * (q ** (k-1)) * (1 - q) / (1 - q ** 3))

for matr in matrD_k:
    print(matr)

[[-12.28571429  0.00142857]
 [ 0.00285714 -0.39428571]]
[[ 4.97658080  0.05796253]
 [ 0.01170960  0.14871194]]
[[ 3.98126464  0.04637002]
 [ 0.00936768  0.11896956]]
[[ 3.18501171  0.03709602]
 [ 0.00749415  0.09517564]]


In [10]:
# Характеристики входного BMAP

matrD_1_ = np.zeros(matrD_k[0].shape)
for matr in matrD_k:
    matrD_1_ += matr
print(matrD_1_)
matr_a = copy.deepcopy(matrD_1_)
for i in range(matr_a.shape[0]):
    matr_a[i][0] = 1

matr_b = np.zeros((matr_a.shape[0], 1))
matr_b[0][0] = 1

matr_a = np.transpose(matr_a)

theta = la.solve(matr_a, matr_b).reshape(-1)

matrdD_1_ = copy.deepcopy(matrD_k[1])

print(theta)

for i in range(2, n+1):
    matrdD_1_ += matrD_k[i] * i
vect_e = np.array([[1.] for i in range(0, matrD_1_.shape[0])])
lamD = np.dot(np.dot(theta, matrdD_1_), vect_e)
print(lamD)

[[-0.14285714  0.14285714]
 [ 0.03142857 -0.03142857]]


[ 0.18032787  0.81967213]
[ 4.69791416]


In [11]:
# Поток поломок MAP
matrH0 = np.array([[-8.110725, 0], [0, -0.26325]])
matrH1 = np.array([[8.0568, 0.053925], [0.146625, 0.116625]])
V_ = matrH1.shape[0]
V = V_ - 1
matrH = matrH0 + matrH1
matr_a = copy.deepcopy(matrH)
for i in range(matr_a.shape[0]):
    print(matr_a)
    matr_a[i][0] = 1
    
matr_b = np.zeros((matr_a.shape[0], 1))
matr_b[0][0] = 1

matr_a = np.transpose(matr_a)

print(matr_a)

gamma = la.solve(matr_a, matr_b).reshape(-1)

vect_e = np.array([[1.] for i in range(0, matrD_1_.shape[0])])
h = np.dot(np.dot(gamma, matrH1), vect_e)
print(h)

[[-0.05392500  0.05392500]
 [ 0.14662500 -0.14662500]]
[[ 1.00000000  0.05392500]
 [ 0.14662500 -0.14662500]]
[[ 1.00000000  1.00000000]
 [ 0.05392500 -0.14662500]]
[ 6.00065225]


In [12]:
# Поток обслуживания PH1
beta1 = np.array([[1, 0]])
matrS1 = np.array([[-20, 20], [0, -20]])
M1 = matrS1.shape[0]
M1_ = M1 + 1
M1_e = np.array([[1], [1]])
matrS1_0 = np.dot(- matrS1, M1_e)
vect_e = np.array([[1.] for i in range(0, matrS1.shape[0])])
print(np.dot(beta1, la.inv(matrS1)))
mu_1 = -la.inv(np.dot(np.dot(beta1, la.inv(matrS1)), vect_e))
print(mu_1)

[[-0.05000000 -0.05000000]]
[[ 10.00000000]]


In [13]:
# Поток обслуживания PH2
beta2 = np.array([[1, 0]])
matrS2 = np.array([[-2, 2], [0, -2]])
M2 = matrS2.shape[0]
M2_ = M2 + 1
M2_e = np.array([[1], [1]])
matrS2_0 = np.dot(- matrS2, M2_e)

vect_e = np.array([[1.] for i in range(0, matrS2.shape[0])])
mu_2 = -la.inv(np.dot(np.dot(beta2, la.inv(matrS2)), vect_e))
print(mu_2)

[[ 1.00000000]]


In [14]:
matrS_w = kron(np.dot(matrS1_0, beta1), 
               np.dot(M2_e, beta2)) + kron(np.dot(M1_e, beta1), np.dot(matrS2_0, beta2))

In [15]:
# Поток переключения с прибора-1 на прибор-2
alpha1 = np.array([[0.05, 0.95]])
matrA1 = np.array([[-1.86075, 0.], [0., -146.9994]])
L1 = matrA1.shape[0]
L1_ = L1 + 1
L1_e = np.array([[1], [1]])
matrA1_0 = - np.dot(matrA1, L1_e)

vect_e = np.array([[1.] for i in range(0, matrA1.shape[0])])
kappa_1 = -la.inv(np.dot(np.dot(alpha1, la.inv(matrA1)), vect_e))
print(kappa_1)

[[ 29.99985287]]


In [16]:
# Поток переключения с прибора-2 на прибор-1
alpha2 = np.array([[0.05, 0.95]])
matrA2 = np.array([[-5.58225, 0.], [0., -440.9982]])
L2 = matrA2.shape[0]
L2_ = L2 + 1
L2_e = np.array([[1], [1]])
matrA2_0 = - np.dot(matrA2, L2_e)

vect_e = np.array([[1.] for i in range(0, matrA1.shape[0])])
kappa_2 = -la.inv(np.dot(np.dot(alpha2, la.inv(matrA2)), vect_e))
print(kappa_2)

[[ 89.99955862]]


In [17]:
# Поток ремонта PH
tau = np.array([[0.98, 0.02]])
matrT = np.array([[-100., 0.], [0., -0.0002]])
T_e = np.array([[1], [1]])
matrT0 = - np.dot(matrT, T_e)

R = matrT.shape[0]
R_ = R + 1
vect_e = np.array([[1.] for i in range(0, matrT.shape[0])])
phi = -np.dot(np.dot(tau, la.inv(matrT)), vect_e)
print(phi)

[[ 100.00980000]]


In [18]:
a = W_ * V_
print(a)

4


In [19]:
print(kronsum(matrD_k[0], matrH0))

[[-20.39643929  0.00000000  0.00142857  0.00000000]
 [ 0.00000000 -12.54896429  0.00000000  0.00142857]
 [ 0.00285714  0.00000000 -8.50501071  0.00000000]
 [ 0.00000000  0.00285714  0.00000000 -0.65753571]]


In [20]:
# Q~0
block00 = kronsum(matrD_k[0], matrH0)
block03 = kron(kron(kron(np.eye(W_), matrH1), tau), alpha1)
block10 = kron(np.eye(a), matrA2_0)
block11 = kronsum(kronsum(matrD_k[0], matrH0), matrA2)
block12 = kron(kron(kron(np.eye(W_), matrH1), tau), L2_e)
block21 = kron(kron(np.eye(a), matrT0), alpha2)
block22 = kronsum(kronsum(matrD_k[0], matrH), matrT)
block30 = kron(kron(np.eye(a), matrT0), L1_e)
block32 = kron(kron(np.eye(a), np.eye(R)), matrA1_0)
block33 = kronsum(kronsum(kronsum(matrD_k[0], matrH), matrT), matrA1)
block01 = np.zeros((block00.shape[0], block11.shape[1]))
block02 = np.zeros((block00.shape[0], block12.shape[1]))
block13 = np.zeros((block10.shape[0], block03.shape[1]))
block20 = np.zeros((block21.shape[0], block10.shape[1]))
block23 = np.zeros((block21.shape[0], block03.shape[1]))
block31 = np.zeros((block30.shape[0], block11.shape[1]))

print(block30.shape)

matrQw_0 = np.bmat([[block00, block01, block02, block03],
                    [block10, block11, block12, block13], 
                    [block20, block21, block22, block23],
                    [block30, block31, block32, block33]])
print(matrQw_0.shape)

(16, 4)
(36, 36)


In [21]:
# Q~k
matrQw_k = [matrQw_0]
for i in range(1, n+1):
    block0 = kron(kron(matrD_k[i], np.eye(V_)), beta1)
    block1 = kron(kron(kron(matrD_k[i], np.eye(V_)), beta2), np.eye(L2))
    block2 = kron(kron(kron(matrD_k[i], np.eye(V_)), beta2), np.eye(R))
    block3 = kron(kron(matrD_k[i], np.eye(V_)), np.eye(R*L1))
    matr_temp = la.block_diag(block0, block1, block2, block3)
    matrQw_k.append(matr_temp)
    
print(matrQw_k[0].shape)
print(matrQw_k[1].shape)

for i in range(matrQw_k[0].shape[0]):
    sum = 0
    for matr in matrQw_k:
        sum += np.sum(matr[i])
    # print('matrQw_k[' + str(i) + '] = ', sum)

(36, 36)
(36, 56)
matrQw_k[0] =  8.881784197e-16
matrQw_k[1] =  8.881784197e-16
matrQw_k[2] =  5.55111512313e-16
matrQw_k[3] =  1.11022302463e-16
matrQw_k[4] =  -4.4408920985e-15
matrQw_k[5] =  -8.881784197e-16
matrQw_k[6] =  2.6645352591e-15
matrQw_k[7] =  -5.77315972805e-14
matrQw_k[8] =  

2.22044604925e-16
matrQw_k[9] =  -1.487698853e-14
matrQw_k[10] =  0.0
matrQw_k[11] =  1.42108547152e-14
matrQw_k[12] =  8.881784197e-16
matrQw_k[13] =  -8.881784197e-16
matrQw_k[14] =  -6.2172489379e-15
matrQw_k[15] =  -8.881784197e-16
matrQw_k[16] =  6.88338275268e-15
matrQw_k[17] =  6.10622663544e-16
matrQw_k[18] =  -8.881784197e-16
matrQw_k[19] =  1.11022302463e-16
matrQw_k[20] =  1.33226762955e-14
matrQw_k[21] =  -1.50990331349e-14
matrQw_k[22] =  8.881784197e-16
matrQw_k[23] =  8.881784197e-16
matrQw_k[24] =  -2.6645352591e-15
matrQw_k[25] =  7.9936057773e-15
matrQw_k[26] =  -8.881784197e-16
matrQw_k[27] =  7.9936057773e-15
matrQw_k[28] =  2.27595720048e-15
matrQw_k[29] =  3.06976666309e-14
matrQw_k[30] =  4.99600361081e-16
matrQw_k[31] =  3.06976666309e-14
matrQw_k[32] =  -5.27355936697e-15
matrQw_k[33] =  -5.27355936697e-15
matrQw_k[34] =  5.55111512313e-17
matrQw_k[35] =  2.31481500634e-14


In [22]:
# Q^0
block0 = kron(np.eye(a), matrS1_0)
block1 = kron(kron(np.eye(a), matrS2_0), np.eye(L2))
block2 = kron(kron(np.eye(a), matrS2_0), np.eye(R))
block3 = np.zeros((a*R*L1, a*R*L1))
matrQv_0 = la.block_diag(block0, block1, block2, block3)

print(matrQv_0.shape)

(56, 36)


In [23]:
# Q_0
block0 = kron(np.eye(a), np.dot(matrS1_0, beta1))
block1 = kron(kron(np.eye(a), np.dot(matrS2_0, beta2)), np.eye(L2))
block2 = kron(kron(np.eye(a), np.dot(matrS2_0, beta2)), np.eye(R))
block3 = np.zeros((a*R*L1, a*R*L1))
matrQ_0 = la.block_diag(block0, block1, block2, block3)

print(matrQ_0.shape)

(56, 56)

In [24]:
# Q_1
block00 = kronsum(kronsum(matrD_k[0], matrH0), matrS1)
block03 = kron(kron(kron(kron(np.eye(W_), matrH1), M1_e), tau), alpha1)
block10 = kron(kron(kron(np.eye(a), M2_e), beta1), matrA2_0)
block11 = kronsum(kronsum(kronsum(matrD_k[0], matrH0), matrS2), matrA2)
block12 = kron(kron(kron(kron(np.eye(W_), matrH1), np.eye(M2)), tau), L2_e)
block21 = kron(kron(kron(np.eye(a), np.eye(M2)), matrT0), alpha1)
block22 = kronsum(kronsum(kronsum(matrD_k[0], matrH), matrS2), matrT)
block30 = kron(kron(kron(np.eye(a), beta1), matrT0), L1_e)
block32 = kron(kron(kron(np.eye(a), beta2), np.eye(R)), matrA1_0)
block33 = kronsum(kronsum(kronsum(matrD_k[0], matrH), matrT), matrA1)
block01 = np.zeros((block00.shape[0], block11.shape[1]))
block02 = np.zeros((block00.shape[0], block12.shape[1]))
block13 = np.zeros((block10.shape[0], block03.shape[1]))
block20 = np.zeros((block21.shape[0], block10.shape[1]))
block23 = np.zeros((block21.shape[0], block03.shape[1]))
block31 = np.zeros((block30.shape[0], block11.shape[1]))

matrQ_1 = np.bmat([[block00, block01, block02, block03], 
                   [block10, block11, block12, block13], 
                   [block20, block21, block22, block23], 
                   [block30, block31, block32, block33]])

print(matrQ_1.shape)

(56, 56)


In [25]:
matrQ_k = [matrQ_0, matrQ_1]
for k in range(2, n+2):
    block0 = kron(matrD_k[k-1], np.eye(V_ * M1))
    block1 = kron(matrD_k[k-1], np.eye(V_*M2*L2))
    block2 = kron(matrD_k[k-1], np.eye(V_*M2*R))
    block3 = kron(matrD_k[k-1], np.eye(V_*R*L1))
    matr_temp = la.block_diag(block0, block1, block2, block3)
    matrQ_k.append(matr_temp)

# for i in range(matrQ_k[0].shape[0]):
#     sum = np.sum(matrQv_0[i])
#     for j in range(1, np.array(matrQ_k).shape[0]):
#         sum += np.sum(matrQ_k[j][i])
#     print('matrQ_k[' + str(i) + '] = ', sum)

matrQ_k[0] =  8.881784197e-16
matrQ_k[1] =  -7.9936057773e-15
matrQ_k[2] =  2.6645352591e-15
matrQ_k[3] =  -8.881784197e-16
matrQ_k[4] =  5.55111512313e-17
matrQ_k[5] =  -1.72084568817e-15
matrQ_k[6] =  1.49880108324e-15
matrQ_k[7] =  1.83186799063e-15
matrQ_k[8] =  8.881784197e-16
matrQ_k[9] =  6.2172489379e-15
matrQ_k[10] =  -2.6645352591e-15
matrQ_k[11] =  6.2172489379e-15
matrQ_k[12] =  4.4408920985e-15
matrQ_k[13] =  -5.77315972805e-14
matrQ_k[14] =  4.4408920985e-15
matrQ_k[15] =  -5.77315972805e-14
matrQ_k[16] =  -8.32667268469e-16
matrQ_k[17] =  -3.36952687974e-14
matrQ_k[18] =  -1.72084568817e-15
matrQ_k[19] =  -3.36952687974e-14
matrQ_k[20] =  -1.66533453694e-16
matrQ_k[21] =  -5.49560397189e-15
matrQ_k[22] =  5.55111512313e-17
matrQ_k[23] =  -5.27355936697e-15
matrQ_k[24] =  6.2172489379e-15
matrQ_k[25] =  -8.881784197e-16
matrQ_k[26] =  6.2172489379e-15
matrQ_k[27] =  -8.881784197e-16
matrQ_k[28] =  -8.881784197e-16
matrQ_k[29] =  -8.881784197e-16
matrQ_k[30] =  -8.88178419

In [26]:
# Проверка генератора Q

In [27]:
# Поиск матрицы G

matrQ_1_neg_inv = np.linalg.inv(-matrQ_k[1])


def calc_G(matrG_prev):
    temp_sum = copy.deepcopy(matrQ_k[0]) 
    for k in range(2, n + 2):
        temp_sum += np.dot(matrQ_k[k], np.linalg.matrix_power(matrG_prev, k))
    matrG_new = np.dot(matrQ_1_neg_inv, temp_sum)
    return matrG_new

matrG_old = np.eye(matrQ_k[1].shape[0])
matrG = calc_G(matrG_old)


i = 1
while la.norm(matrG - matrG_old, ord=np.inf) >= eps_G:
    matrG_old = matrG
    matrG = calc_G(matrG_old)
    i += 1
print(i)
print(la.norm(matrG, ord=np.inf))

372
1.0000000017054


In [28]:
la.norm(matrG, ord=np.inf)

1.0000000017054

In [29]:
temp_sum = copy.deepcopy(matrQ_k[1])
for k in range(2, n+2):
    temp_sum += np.dot(matrQ_k[k], np.linalg.matrix_power(matrG, k-1))
matrG_0 = la.inv(temp_sum)
matrG_0 = np.dot(-matrG_0, matrQv_0)
# pd.DataFrame(matrG_0)

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,0.396833,0.002519,0.268151,0.244523,7.445429e-05,2.186799e-05,1.023466e-06,1.736665e-07,0.000140,1.769089e-05,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.606271,0.001715,0.175245,0.159389,5.161550e-05,1.516101e-05,7.092874e-07,1.203822e-07,0.000092,1.156039e-05,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.004480,0.483810,0.167202,0.289445,2.298020e-06,6.712217e-07,2.023818e-06,3.154654e-07,0.000088,1.116733e-05,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.002866,0.695133,0.098474,0.171100,1.484291e-06,4.338016e-07,1.315272e-06,2.050693e-07,0.000052,6.577608e-06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000589,0.000016,0.968647,0.009691,1.877517e-07,3.480540e-08,4.267502e-09,2.856422e-10,0.000491,6.273390e-05,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000387,0.000009,0.981642,0.005646,1.142628e-07,2.160944e-08,2.560749e-09,1.769470e-10,0.000288,3.687099e-05,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.000021,0.000694,0.015557,0.982532,8.357641e-09,1.808865e-09,7.072446e-09,4.638857e-10,0.000016,1.987886e-06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.000011,0.000412,0.007820,0.991163,4.352251e-09,9.521438e-10,3.727694e-09,2.568525e-10,0.000008,1.004572e-06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.212777,0.003023,0.346482,0.317741,5.306661e-03,1.104153e-04,2.297095e-06,8.226630e-07,0.000183,2.290504e-05,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.387562,0.002553,0.272251,0.248283,7.791672e-05,4.171573e-05,1.067953e-06,1.828495e-07,0.000142,1.796158e-05,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
matrQ_il = []
for i in range(0, p_num):
    matrQ_il.append([])
    if i == 0:
        for l in range(0, n + 1):
            # здесь до n, т.к. нет больше матриц Q_k
            temp_matr = matrQw_k[l].copy()
            for k in range(l + 1, n + 1):
                mult_matr = matrQw_k[k].copy()
                for kk in range(k - 1, l - 1, -1):
                    if kk == 0:
                        mult_matr = mult_matr.dot(matrG_0)
                    else:
                        mult_matr = mult_matr.dot(matrG)
                print("mult matr 0 shape : ", mult_matr.shape)
                temp_matr += mult_matr
            matrQ_il[i].append(temp_matr)
        for l in range(n + 1, p_num):
            matrQ_il[i].append(np.zeros(matrQw_k[1].shape))
    else:
        for l in range(0, p_num):
            if l >= i and (l - i) <= (n + 1):
                if (l - i + 1) <= (n + 1): 
                    temp_matr = copy.deepcopy(matrQ_k[l - i + 1])
                else:
                    temp_matr = np.zeros(matrQ_k[0].shape)
                    
                for k in range(l + 1, p_num):  # sum from l+1 to inf
                    if (k - i) <= (n + 1):
                        mult_matr = copy.deepcopy(matrQ_k[k - i])
                        for kk in range(l, k):
                            mult_matr = np.dot(mult_matr, matrG)
                        
                        temp_matr += mult_matr
                matrQ_il[i].append(temp_matr)
            else:
                matrQ_il[i].append(np.zeros(matrQ_k[0].shape))

# print(len(matrQ_il[3]))
# print(matrQ_il[4][4].shape)

mult matr 0 shape :  (36, 36)
mult matr 0 shape :  (36, 36)
mult matr 0 shape :  (36, 36)
mult matr 0 shape :  (36, 56)
mult matr 0 shape :  (36, 56)
mult matr 0 shape :  (36, 56)


(56, 56)


In [31]:
matrPhi_0 = np.eye(matrQ_il[0][0].shape[0])
matrPhi_l = [matrPhi_0]
for l in range(1, p_num):
    temp_matr = np.dot(np.dot(matrPhi_l[0], matrQ_il[0][l]), la.inv(-matrQ_il[l][l]))
    for i in range(1, l):
        # print(matrPhi_l[i].dot(matrQ_il[i][l]).dot(la.inv(-matrQ_il[l][l])).shape)
        temp_matr += np.dot(np.dot(matrPhi_l[i], matrQ_il[i][l]), la.inv(-matrQ_il[l][l]))
    matrPhi_l.append(temp_matr)

In [32]:
print(la.norm(matrPhi_l[p_num - 1], ord=np.inf))

1.9063345073976335e-05


In [33]:
# Вычисление p_0
matr_a = np.array(- copy.deepcopy(matrQ_il[0][0]))
vect_eaR = np.array([[1.] for _ in range(matrPhi_l[0].shape[1])])
for i in range(p_num):
    vect_e = np.array([[1.] for _ in range(matrPhi_l[i].shape[1])])
    vect_eaR += np.dot(matrPhi_l[i], vect_e)

for i in range(matr_a.shape[0]):
    matr_a[i][0] = vect_eaR[i][0] + 1.

matr_b = np.zeros((matr_a.shape[0], 1))
matr_b[0][0] = 1.
matr_a = np.transpose(matr_a)
p0 = np.transpose(la.solve(matr_a, matr_b))

# print(p0)
print(np.sum(p0))

          0         1         2         3             4             5   \
0  17.593418 -0.027848 -4.270317 -3.814587 -5.283298e-04 -1.545589e-04   
1  -0.047409  8.846302 -2.761106 -4.865908 -2.025012e-05 -5.866715e-06   
2  -0.009719 -0.000076  8.152542 -0.015632 -1.321000e-06 -3.792060e-07   
3  -0.000125 -0.011904 -0.016705  0.295017 -5.243615e-08 -1.487570e-08   
4  -7.093302 -0.029039 -4.811932 -4.343479  2.595208e+01 -5.767135e-04   

             6             7         8         9     ...           26  \
0 -7.343893e-06 -1.233945e-06 -0.002230 -0.000282    ...    -0.000054   
1 -1.565337e-05 -2.420653e-06 -0.001461 -0.000184    ...    -0.000117   
2 -1.901858e-08 -3.033235e-09 -0.000180 -0.000023    ...    -0.000000   
3 -3.982019e-08 -5.920497e-09 -0.000011 -0.000001    ...    -0.000000   
4 -1.280842e-05 -4.316317e-06 -0.004911 -0.000336    ...    -0.000000   

         27        28        29        30        31        32        33  \
0 -0.001025 -0.000000 -0.000000 -0.000000

In [40]:
vect_p_l = [p0]
p_sum = np.sum(p0)
for l in range(1, p_num):
    vect_p_l.append(vect_p_l[0].dot(matrPhi_l[l]))
    print(np.sum(vect_p_l[l]))
print(p_sum)

0.0300186236609
0.0237768214161
0.0166589775773
0.00880335673372
0.00619954662819
0.00434275519239
0.00315535515806
0.00251443381411
0.00209638519347
0.00182461525673
0.00164141229084
0.00150634891915
0.00140099632465
0.00131390205306
0.00123820969417
0.00117022141329
0.00110771864292
0.0010493747366
0.000994415455159
0.000942354829918
0.000892879791721
0.000845781527213
0.000800908738811
0.000758145096471
0.000717395315505
0.000678576326208
0.000641612676339
0.000606433666516
0.000572971628346
0.000541161007765
0.000510937831808
0.000482239433912
0.000455004344062
0.000429172264365
0.000404684097998
0.00038148200685
0.000359509480532
0.000338711407991
0.000319034145495
0.000300425577168
0.000282835166171
0.000266213995533
0.000250514798378
0.00023569197778
0.000221701616783
0.000208501479295
0.00019605100269
0.000184311282981
0.000173245053445
0.00016281665756
0.000152992017064
0.000143738595928
0.000135025360922
0.000126822739456
0.000119102575275
0.00011183808254
0.00010500379877
9.

In [35]:
# Условие эргодичности
block00 = kronsum(matrH0, matrS1) + kron(np.eye(V_), matrS1_0.dot(beta1))
block03 = kron(kron(kron(matrH1, M1_e), tau), alpha1)
block10 = kron(kron(np.eye(V_), M2_e.dot(beta1)), matrA2_0)
block11 = kronsum(kronsum(matrH0, matrS2), matrA2) + kron(kron(np.eye(V_), matrS2_0.dot(beta2)), np.eye(L2))
block12 = kron(kron(kron(matrH1, np.eye(M2)), tau), L2_e)
block21 = kron(kron(kron(np.eye(V_), np.eye(M2)), matrT0), alpha1)
block22 = kronsum(kronsum(matrH, matrS2), matrT) + kron(kron(np.eye(V_), matrS2_0.dot(beta2)), np.eye(R))
block30 = kron(kron(kron(np.eye(V_), beta1), matrT0), L1_e)
block32 = kron(kron(kron(np.eye(V_), beta2), np.eye(R)), matrA1_0)
block33 = kronsum(kronsum(matrH, matrT), matrA1)
block01 = np.zeros((block00.shape[0], block11.shape[1]))
block02 = np.zeros((block00.shape[0], block12.shape[1]))
block13 = np.zeros((block10.shape[0], block03.shape[1]))
block20 = np.zeros((block21.shape[0], block00.shape[1]))
block23 = np.zeros((block21.shape[0], block03.shape[1]))
block31 = np.zeros((block30.shape[0], block11.shape[1]))
matrGamma = np.bmat([[block00, block01, block02, block03],
                     [block10, block11, block12, block13],
                     [block20, block21, block22, block23],
                     [block30, block31, block32, block33]])
pd.DataFrame(matrGamma)

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,-28.110725,20.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.000000,0.000000,0.394783,7.500881,0.008057,0.153079,0.002642,0.050204,0.000054,0.001025
1,20.000000,-28.110725,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.000000,0.000000,0.394783,7.500881,0.008057,0.153079,0.002642,0.050204,0.000054,0.001025
2,0.000000,0.000000,-20.26325,20.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.000000,0.000000,0.007185,0.136508,0.000147,0.002786,0.005715,0.108578,0.000117,0.002216
3,0.000000,0.000000,20.00000,-20.26325,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.000000,0.000000,0.007185,0.136508,0.000147,0.002786,0.005715,0.108578,0.000117,0.002216
4,5.582250,0.000000,0.00000,0.00000,-15.692975,0.000000,2.000000,0.000000,0.00000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,440.998200,0.000000,0.00000,0.00000,0.000000,-451.108925,0.000000,2.000000,0.00000,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,5.582250,0.000000,0.00000,0.00000,2.000000,0.000000,-15.692975,0.000000,0.00000,0.00000,...,0.052846,0.001079,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,440.998200,0.000000,0.00000,0.00000,0.000000,2.000000,0.000000,-451.108925,0.00000,0.00000,...,0.052846,0.001079,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,5.58225,0.00000,0.000000,0.000000,0.000000,0.000000,-7.84550,0.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,440.99820,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,-443.26145,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [35]:
matr_a = copy.deepcopy(matrGamma)
for i in range(matr_a.shape[0]):
    matr_a[i][0] = 1
matr_b = np.zeros((matr_a.shape[0], 1))
matr_b[0][0] = 1
matr_a = np.transpose(matr_a)

x = la.solve(matr_a, matr_b).reshape(-1)

print('x = ', x)
x1 = x[0:V_*M1]
x2 = x[V_*M1:V_*M1 + V_*M2*L2]
x3 = x[V_*M1 + V_*M2*L2: V_*M1 + V_*M2*L2 + V_*M2*R]

print('x1 = ', x1)
print('x2 = ', x2)
print('x3 = ', x3)

LinAlgError: singular matrix

In [ ]:
e_V_ = np.array([[1.] for i in range(0, V_)])
e_R = np.array([[1.] for i in range(0, R)])
pi1 = x1.dot(kron(e_V_, np.eye(M1)))
pi2 = x2.dot(kron(kron(e_V_, np.eye(M2)), L2_e))
pi3 = x3.dot(kron(kron(e_V_, np.eye(M2)), e_R))
print('pi1 = ', pi1)
print('pi2 = ', pi2)
print('pi3 = ', pi3)

In [ ]:
rho = pi1.dot(matrS1_0) + (pi2 + pi3).dot(matrS2_0)
rho = rho.tolist()[0][0]
print('rho = ', rho)

In [ ]:
print(la.norm(np.eye(matrQ_il[0][0].shape[0]).dot(matrQ_il[0][1]).dot(la.inv(-matrQ_il[1][1])), np.inf))

In [ ]:
result = np.sum(matrQv_0[0])
i = 0
for matrQk in matrQ_k:
    if i > 0:
        result += np.sum(matrQk[0])
    i += 1
print(result)